<a href="https://colab.research.google.com/github/Rohan-damagatla/LetsUpgradeDataScience/blob/main/Datakathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost #Install catboost on colab

In [ ]:
#Reading input data and Onehotencoding the categorical columns
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler

df = pd.read_excel('/content/train.xlsx')

primarycol = 'JOBID'
depcol = 'ELAPSEDTIME'
indcol = list(df.columns)
indcol.remove(primarycol)
indcol.remove(depcol)
    
independentcols=list(df.columns)
removecols=[depcol,primarycol]
independentcols = [n for n in independentcols if n not in removecols]

encoded={}
cat_col=[]
for col, col_type in df.dtypes.iteritems():
      if col_type == 'O':
          cat_col.append(col)
cat_col = [x for x in cat_col if x in indcol]
for i in range(len(cat_col)):
    encoder = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(encoder.fit_transform(df[[cat_col[i]]]).toarray())
    enc_df.columns = encoder.get_feature_names([cat_col[i]])
    df=df.drop(cat_col[i], axis=1)
    df=pd.concat([df, enc_df], axis=1)
    encoded.update({cat_col[i]:encoder})
    
independentcols=list(df.columns)
removecols=[depcol,primarycol]
independentcols = [n for n in independentcols if n not in removecols]

x=df[independentcols].values
y=df[depcol].values

In [ ]:
#Applying log tranformation on the output column
from scipy import stats
from scipy.stats import norm, skew
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(y , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(y)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(y, plot=plt)
plt.show()

y = np.log(y)

#Check the new distribution 
sns.distplot(y , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(y)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(y, plot=plt)
plt.show()

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

#Define the cross validation score
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x)
    rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

#Define gradient boost with best obtained params
GBoost = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.2,
                                   max_depth=10, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

#Define XGboost with best params
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.15, max_depth=5, 
                             min_child_weight=1.7817, n_estimators=2000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

#Define Lightgbm with best params
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,max_depth=10,
                              learning_rate=0.3, n_estimators=5000,
                              max_bin = 55, bagging_fraction = 0.9,
                              bagging_freq = 5, feature_fraction = 0.9,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

#Define Lightgbm with best params
model_cat = CatBoostRegressor(verbose=0,depth= 10, l2_leaf_reg = 1, iterations = 5000, learning_rate = 0.1)

#Form a stacking average model using the above models as base and meta models
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

stacked_averaged_models = StackingAveragedModels(base_models = (GBoost, model_xgb,model_cat),
                                                 meta_model = model_lgb)

In [ ]:
#Read test data and apply the same encoder for categorical columns as train data
test = pd.read_excel('/content/test1.xlsx')

for i in range(len(cat_col)):
	enc_df = pd.DataFrame(encoded[cat_col[i]].transform(test[[cat_col[i]]]).toarray())
	enc_df.columns = encoded[cat_col[i]].get_feature_names([cat_col[i]])
	test=test.drop(cat_col[i], axis=1)
	test=pd.concat([test, enc_df], axis=1)
test_x=test[independentcols].values

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#Modelling using train data and predicting on test data
stacked_averaged_models.fit(x, y)
stacked_train_pred = stacked_averaged_models.predict(x)
stacked_pred = np.exp(stacked_averaged_models.predict(test_x))
print(rmsle(y, stacked_train_pred))

model_cat.fit(x, y)
cat_train_pred = model_cat.predict(x)
cat_pred = np.exp(model_cat.predict(test_x))
print(rmsle(y, cat_train_pred))


GBoost.fit(x, y)
grad_train_pred = GBoost.predict(x)
grad_pred = np.exp(GBoost.predict(test_x))
print(rmsle(y, grad_train_pred))

In [ ]:
#Weighted values for predicting the output
ensemble = cat_pred*0.49 + stacked_pred*0.5 + grad_pred*0.05
test['PREDICTION'] = ensemble

test = test[[primarycol,'PREDICTION']]
test.to_csv('data_cbgridcvgb.csv', sep=';', index=False)

from google.colab import files
files.download('data_cbgridcvgb.csv')